In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import t, norm
from joblib import Parallel, delayed

# Load data
data = pd.read_csv('data.csv')

# Constants
num_obvs = 100_000
r = np.array([.295, .49, .41, .415, .338, .64, .403, .476])
sector_indices = data['sector'].values
sec_loading = r[sector_indices]
datat = norm.ppf(data['p'])
shift = 2.5  # Mean shift for importance sampling

# Generate importance sampling factors
def generate_importance_samples(num_obvs, dim, shift):
    factors = np.random.normal(shift, 1, (num_obvs, dim))  # Shifted sampling
    weights = norm.pdf(factors, 0, 1) / norm.pdf(factors, shift, 1)  # Importance weights
    return factors, weights

# Process observations with importance sampling
def process_obs_with_weights(obs, weight):
    m_factor = obs[0]
    sec_factor = obs[:len(r)][sector_indices]
    res_factor = obs[len(r):]

    control_variate = (
            r[0]**0.5 * m_factor
            + (sec_loading - r[0])**0.5 * sec_factor
            + (1 - sec_loading)**0.5 * res_factor
    )

    ind = control_variate < datat
    loss = np.zeros(len(data))

    if np.any(ind):
        loss[ind] = (
                data.loc[ind, 'm'].values
                + data.loc[ind, 'd'].values * np.clip(t.rvs(df=3, size=sum(ind)), -5, 5)
        )

    # Control variate adjustment
    expected_control = np.mean(control_variate)
    alpha = np.cov(loss, control_variate)[0, 1] / np.var(control_variate)
    loss -= alpha * (control_variate - expected_control)

    # Apply importance weight to loss
    weighted_loss = weight * np.sum(loss)
    return weighted_loss, weight * np.var(loss)

# Generate factors and weights
factors, weights = generate_importance_samples(num_obvs, len(r) + len(data), shift)
antithetic_factors = -factors  # Antithetic sampling
combined_factors = np.vstack([factors, antithetic_factors])
combined_weights = np.hstack([weights, weights])  # Combine weights for antithetic samples

# Parallel processing with weights
answers = Parallel(n_jobs=-1, verbose=5)(
    delayed(process_obs_with_weights)(combined_factors[i], combined_weights[i])
    for i in range(len(combined_factors))
)
answers = np.array(answers)

# Extract weighted results
sample_losses = answers[:, 0]
sample_vars = answers[:, 1]

# Calculate Value-at-Risk (VaR) with weighted losses
VaR_importance = np.percentile(-np.array(sample_losses), 99.9)

# Output Results
print(f"Value-at-Risk (99.9%) with Importance Sampling: {VaR_importance:.5f}")
print(f"Mean Weighted Loss: {np.mean(sample_losses):.5f}")
print(f"Mean Weighted Variance: {np.mean(sample_vars):.5f}")
